# RFM Analysis
#### Lets import the required libraries first

In [1]:
import numpy as np
import pandas as pd

#### Lets read the dataset 

In [2]:
df = pd.read_csv('D:/Datasets/CustomerData/data2.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom


In [3]:
print('There are {} rows and {} columns'.format(df.shape[0], df.shape[1]))

There are 541909 rows and 8 columns


In [4]:
df.isnull().mean()

InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

#### 'Description' and 'CustomerID' have some NaN values. Lets drop the rows which have missing 'CustomerID'. For RFM analysis we will focus on three things i.e. Recency, Frequency and Monetization. So we dont need to fill the NaN values of Description

In [5]:
df = df.dropna(axis=0, subset=['CustomerID'])

#### KPIs will be as follow
1. Recency: Date of Last Purchase
2. Frequency: Total number of orders
3. Monetization: Total OrderValue

#### We have Unit Price and Quantity so lets calculate Total Order Value

In [6]:
df['OrderValue'] = df['Quantity'] * df['UnitPrice']

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,OrderValue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom,20.34


#### Lets find number of days between Last Purchase and today 

In [8]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [9]:
from datetime import date
today = date.today()
tod = pd.to_datetime(today)

In [10]:
df['DaysFromLastPurchase'] = tod - df['InvoiceDate']

In [11]:
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,OrderValue,DaysFromLastPurchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,3709 days 15:34:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,3709 days 15:34:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,3709 days 15:34:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,3709 days 15:34:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,3709 days 15:34:00
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,15.30,3709 days 15:34:00
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50,3709 days 15:34:00
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10,3709 days 15:32:00
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10,3709 days 15:32:00
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08,3709 days 15:26:00


#### Now lets create a new dataframe with grouping customerID. We will add up the quantity and total order value of same customerID by using groupby 

In [12]:
newdf = df.groupby('CustomerID')['Quantity','OrderValue'].sum()

<ipython-input-12-64aebc6e905f>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  newdf = df.groupby('CustomerID')['Quantity','OrderValue'].sum()


In [13]:
newdf.head()

,Quantity,OrderValue
CustomerID,,
12346.0,0,0.00
12347.0,2458,4310.00
12348.0,2341,1797.24
12349.0,631,1757.55
12350.0,197,334.40


#### We will add a new column DaysFromLastPurchase and initialise it with recent purchase of that particular customer. We do this by grouping CustomerID and picking the min value from DaysFromLastPurchase of df dataframe . So it will give number of days from recent purchase of the particular customer 

In [14]:
newdf['DaysFromLastPurchase'] = df.groupby('CustomerID')['DaysFromLastPurchase'].min()

In [15]:
newdf = newdf.reset_index()

In [16]:
newdf.head()

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase
0,12346.0,0,0.00,3661 days 13:43:00
1,12347.0,2458,4310.00,3338 days 08:08:00
2,12348.0,2341,1797.24,3411 days 10:47:00
3,12349.0,631,1757.55,3354 days 14:09:00
4,12350.0,197,334.40,3646 days 07:59:00


#### Adding three new columns of three KPIs

In [17]:
newdf['Recency'] = ""
newdf['Frequency'] = ""
newdf['Monetization'] = ""

In [18]:
newdf.head(10)

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
0,12346.0,0,0.00,3661 days 13:43:00,,,
1,12347.0,2458,4310.00,3338 days 08:08:00,,,
2,12348.0,2341,1797.24,3411 days 10:47:00,,,
3,12349.0,631,1757.55,3354 days 14:09:00,,,
4,12350.0,197,334.40,3646 days 07:59:00,,,
5,12352.0,470,1545.41,3372 days 09:23:00,,,
6,12353.0,20,89.00,3540 days 06:13:00,,,
7,12354.0,530,1079.40,3568 days 10:49:00,,,
8,12355.0,240,459.40,3550 days 10:11:00,,,
9,12356.0,1591,2811.43,3358 days 15:20:00,,,


In [19]:
newdf.shape

(4372, 7)

#### Now lets fill the Recency column first. We will sort the dataframe according to DaysFromLastPurchase in ascending order. After that we will score them accordingly. There are 4372 rows so I am dividing it into 4 quarters and scoring 1 to first quarter of rows , then 2 to the second quarter of rows and likewise..

In [20]:
newdf.sort_values(['DaysFromLastPurchase'], axis=0, ascending=True, inplace=True)

In [21]:
newdf.head(10)

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
275,12680.0,443,862.81,3336 days 11:10:00,,,
587,13113.0,2594,10510.00,3336 days 11:11:00,,,
2562,15804.0,2515,3848.55,3336 days 11:29:00,,,
1067,13777.0,12804,25748.35,3336 days 11:35:00,,,
3854,17581.0,5861,10736.11,3336 days 11:39:00,,,
330,12748.0,24210,29072.10,3336 days 11:40:00,,,
301,12713.0,508,848.55,3336 days 11:44:00,,,
146,12526.0,632,1316.66,3336 days 11:51:00,,,
3215,16705.0,5458,13946.13,3336 days 11:52:00,,,
2192,15311.0,37720,59419.34,3336 days 12:00:00,,,


In [22]:
newdf.iloc[:1093, -3] = 1

In [23]:
newdf.head()

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
275,12680.0,443,862.81,3336 days 11:10:00,1,,
587,13113.0,2594,10510.00,3336 days 11:11:00,1,,
2562,15804.0,2515,3848.55,3336 days 11:29:00,1,,
1067,13777.0,12804,25748.35,3336 days 11:35:00,1,,
3854,17581.0,5861,10736.11,3336 days 11:39:00,1,,


In [24]:
newdf.iloc[1093:2186, -3] = 2
newdf.iloc[2186:3279, -3] = 3
newdf.iloc[3279: , -3] = 4

In [25]:
newdf.tail()

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
3129,16583.0,111,233.45,3709 days 11:57:00,4,,
4096,17908.0,173,243.28,3709 days 12:15:00,4,,
359,12791.0,97,192.60,3709 days 12:33:00,4,,
1046,13747.0,8,79.60,3709 days 13:23:00,4,,
4212,18074.0,190,489.60,3709 days 14:07:00,4,,


#### Now lets initialise Frequency column similarly. This time we will score according to descending order of Quantity column

In [26]:
newdf.sort_values(['Quantity'], axis=0, ascending=False, inplace=True)
newdf.head()

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
1703,14646.0,196719,279489.02,3337 days 11:48:00,1,,
55,12415.0,77242,123725.45,3360 days 09:38:00,2,,
1895,14911.0,77180,132572.62,3337 days 08:06:00,1,,
3758,17450.0,69029,187482.17,3344 days 10:31:00,1,,
4233,18102.0,64122,256438.49,3336 days 12:10:00,1,,


In [27]:
newdf.iloc[:1093, -2] = 1
newdf.iloc[1093:2186, -2] = 2
newdf.iloc[2186:3279, -2] = 3
newdf.iloc[3279: , -2] = 4

In [28]:
newdf.head()

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
1703,14646.0,196719,279489.02,3337 days 11:48:00,1,1,
55,12415.0,77242,123725.45,3360 days 09:38:00,2,1,
1895,14911.0,77180,132572.62,3337 days 08:06:00,1,1,
3758,17450.0,69029,187482.17,3344 days 10:31:00,1,1,
4233,18102.0,64122,256438.49,3336 days 12:10:00,1,1,


#### Now next we initialize the monetization column similarly. This time we will score according to Order value in descending order

In [29]:
newdf.sort_values(['OrderValue'], axis=0, ascending=False, inplace=True)
newdf.head()

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
1703,14646.0,196719,279489.02,3337 days 11:48:00,1,1,
4233,18102.0,64122,256438.49,3336 days 12:10:00,1,1,
3758,17450.0,69029,187482.17,3344 days 10:31:00,1,1,
1895,14911.0,77180,132572.62,3337 days 08:06:00,1,1,
55,12415.0,77242,123725.45,3360 days 09:38:00,2,1,


In [30]:
newdf.iloc[:1093, -1] = 1
newdf.iloc[1093:2186, -1] = 2
newdf.iloc[2186:3279, -1] = 3
newdf.iloc[3279: , -1] = 4

In [31]:
newdf.head(10)

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
1703,14646.0,196719,279489.02,3337 days 11:48:00,1,1,1
4233,18102.0,64122,256438.49,3336 days 12:10:00,1,1,1
3758,17450.0,69029,187482.17,3344 days 10:31:00,1,1,1
1895,14911.0,77180,132572.62,3337 days 08:06:00,1,1,1
55,12415.0,77242,123725.45,3360 days 09:38:00,2,1,1
1345,14156.0,57025,113384.14,3345 days 13:06:00,1,1,1
3801,17511.0,63012,88125.38,3338 days 13:48:00,1,1,1
3202,16684.0,49390,65892.08,3340 days 09:54:00,1,1,1
1005,13694.0,61803,62653.10,3339 days 14:28:00,1,1,1
2192,15311.0,37720,59419.34,3336 days 12:00:00,1,1,1


In [32]:
newdf.tail(10)

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
264,12666.0,-56,-227.44,3695 days 09:18:00,4,4,4
2892,16252.0,-158,-295.09,3702 days 07:45:00,4,4,4
2560,15802.0,-1,-451.42,3478 days 12:20:00,3,4,4
619,13154.0,-1,-611.86,3480 days 08:39:00,4,4,4
2578,15823.0,-283,-840.76,3673 days 11:02:00,4,4,4
125,12503.0,-1,-1126.00,3673 days 12:09:00,4,4,4
3870,17603.0,-31,-1165.30,3386 days 04:41:00,2,4,4
1384,14213.0,-244,-1192.20,3707 days 14:13:00,4,4,4
2236,15369.0,-1,-1592.49,3480 days 08:49:00,4,4,4
3756,17448.0,-1,-4287.63,3480 days 11:34:00,4,4,4


#### Lets sort back according to index and see our result

In [33]:
result = newdf.sort_index()
result.head(50)

,CustomerID,Quantity,OrderValue,DaysFromLastPurchase,Recency,Frequency,Monetization
0,12346.0,0,0.00,3661 days 13:43:00,4,4,4
1,12347.0,2458,4310.00,3338 days 08:08:00,1,1,1
2,12348.0,2341,1797.24,3411 days 10:47:00,3,1,1
3,12349.0,631,1757.55,3354 days 14:09:00,2,2,1
4,12350.0,197,334.40,3646 days 07:59:00,4,3,3
5,12352.0,470,1545.41,3372 days 09:23:00,2,2,2
6,12353.0,20,89.00,3540 days 06:13:00,4,4,4
7,12354.0,530,1079.40,3568 days 10:49:00,4,2,2
8,12355.0,240,459.40,3550 days 10:11:00,4,3,3
9,12356.0,1591,2811.43,3358 days 15:20:00,2,1,1
